<a href="https://colab.research.google.com/github/jorgeulisses/blog-serverless-cpp/blob/main/C%C3%B3pia_de_faceless_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building AI-Powered Faceless Videos with VideoDB Editor
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/editor/creative/faceless_video.ipynb)

---

Faceless videos are everywhere—TikTok, YouTube Shorts, Instagram Reels. They combine engaging visuals with voiceover narration and captions, but never show a person on camera. Think gaming clips with commentary, stock footage with educational content, or animated explainers.

In this notebook, we'll build a complete faceless video pipeline using VideoDB Editor. We'll:
- Generate an engaging script from a topic
- Convert that script to natural voiceover
- Layer it with background visuals and music

Let's create content programmatically.

## Installing VideoDB Editor SDK

First, we'll install the VideoDB SDK.

In [ ]:
%pip -q install videodb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 5.0 MB/s eta 0:00:00


## Connecting to VideoDB

We'll establish a secure connection using your API key.

In [ ]:
import videodb
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key:" )
os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
print("✓ Connected to VideoDB securely!")

Please enter your VideoDB API Key:··········
✓ Connected to VideoDB securely!


Now let's get our collection where we'll store and manage assets.

In [ ]:
coll = conn.get_collection()
print(f"✓ Collection ready: {coll.id}")

✓ Collection ready: c-b2bb2a97-d325-4cc3-a7ea-a75de0227e58


## Uploading Background Video

Faceless videos need engaging visuals. We'll upload a background video (gaming footage, stock video, or b-roll) that will play silently while our AI voiceover narrates.

In [ ]:
# Upload background video
background_video = coll.upload(url='https://www.youtube.com/watch?v=VL1CHvsUSNo')
print(f"✓ Uploaded background video: {background_video.id}")

# To re-run without uploading again, uncomment:
# background_video = coll.get_video("your_video_id")

✓ Uploaded background video: m-z-019ca51a-4195-79d1-893c-c0ce3df67a84


### Checking the uploaded asset

In [ ]:
from videodb import play_stream
play_stream(background_video.generate_stream())

## Uploading Background Music

Background music adds energy and polish. We'll keep it at a low volume so it doesn't overpower the voiceover.

In [ ]:
from videodb import MediaType

# Upload background music as audio
background_music = conn.upload(
    url='https://www.youtube.com/watch?v=kkoIpjQ16YY',
    media_type=MediaType.audio
)
print(f"✓ Uploaded background music: {background_music.id}")

# To re-run without uploading again, uncomment:
# background_music = coll.get_audio("your_audio_id")

✓ Uploaded background music: a-z-019ca51b-f3c7-77e3-a7ad-b40d72077128


## Creating Our Video Topic

Let's choose a topic for our faceless video. This could be anything—tech trends, gaming insights, life hacks, or educational content.

In [ ]:
# Get topic from user
video_topic = input("Enter your video topic : ")
print(f"\n✓ Topic set: {video_topic}")

Enter your video topic : Artificial Inteligence

✓ Topic set: Artificial Inteligence


## Generating the Script with AI

Now we'll use AI to write an engaging script. We're targeting a short, punchy script (around 300-400 words for a 2-minute video) with a fun, GenZ creator vibe—perfect for social media.

In [ ]:
# Create prompt for script generation
script_prompt = f"""You are a GenZ content creator writing a script for a faceless video about: "{video_topic}"

Your task: Write an engaging, fun, fast-paced voiceover script.

Style Guidelines:
- Conversational and energetic tone (like you're talking to a friend)
- Use short, punchy sentences
- Include hooks and interesting facts
- Keep it engaging and easy to follow
- No intros like "Hey guys" or "In this video" - jump straight into the content
- Length: 300-400 words (about 2 minutes when spoken)

Critical: Return ONLY the script text. No titles, no commentary, no explanations. Just the pure voiceover script."""

# Generate script using AI
print("Generating script...")
script_response = coll.generate_text(
    prompt=script_prompt,
    response_type="text",
    model_name="pro"
)

script = script_response.get('output', script_response)
print("\n✓ Script generated!\n")
print("=" * 60)
print(script)
print("=" * 60)

Generating script...

✓ Script generated!

You know that Spotify playlist that just *gets* your soul? Or how Netflix is low-key psychic and knows exactly what you want to binge next? Yeah, that’s not magic. That’s AI, and it’s already running your life.

Okay, so what even *is* Artificial Intelligence?
Forget the complicated stuff. Think of it like giving a computer a brain.
Not a squishy, human one, but a massive digital brain made of pure code.
We're teaching machines to learn from tons of data, spot patterns, and make decisions on their own.
It's not about scary robots from the movies. Well, not yet anyway.
It's about making our tech work *for* us, in ways we never thought possible.

You're literally using it all day and probably don't even realize it.
That TikTok filter that turns you into a manga character? AI.
Asking Alexa to set a 10-minute timer for your pizza? AI.
Your phone's camera automatically making your food pics look fire? You guessed it. AI.
It’s the ultimate digital g

## Converting Script to Voiceover

Let's turn that script into natural-sounding speech using AI voice generation.

In [ ]:
# Generate AI voiceover from script
print("Generating voiceover...")
voiceover_audio = coll.generate_voice(
    text=script,
    voice_name="Default"
)
print(f"✓ Voiceover generated: {voiceover_audio.id}")

Generating voiceover...
✓ Voiceover generated: a-z-019ca51e-da41-7860-b8f6-84f4ad1295b4


## Building the Video: Importing Editor Components

Now we have all our assets. Let's start composing them into a final video using the Editor.

In [ ]:
from videodb.editor import (
    Timeline, Track, Clip,
    VideoAsset, AudioAsset, CaptionAsset,
    CaptionAnimation, CaptionAlignment, Positioning, FontStyling
)
from videodb import play_stream

print("✓ Editor components imported")

✓ Editor components imported


## Layer 1: Background Video (Muted)

Let's start by creating our timeline and adding the background video. We'll mute it (volume=0) since we only want the visuals. We're using a 60-second duration for this demo.

In [ ]:
# Initialize timeline
timeline = Timeline(conn)
timeline.background = "#000000"  # Black background

# Create video clip (muted)
video_clip = Clip(
    asset=VideoAsset(
        id=background_video.id,
        start=3,
        volume=0  # Muted - we only want visuals
    ),
    duration=float(voiceover_audio.length)
)

# Add to first track
video_track = Track()
video_track.add_clip(0, video_clip)
timeline.add_track(video_track)

print("✓ Background video track added")
print("Generating stream...")
stream_url = timeline.generate_stream()
play_stream(stream_url)

✓ Background video track added
Generating stream...


We can see our background video playing silently. Next, let's add the audio layer.

## Layer 2: Audio Voiceover

Now we'll add a second track with the AI Generated Voiceover:


In [ ]:
# Create voiceover clip (normal volume)
voiceover_clip = Clip(
    asset=AudioAsset(
        id=voiceover_audio.id,
        start=0,
        volume=1.0  # Full volume for clear narration
    ),
    duration = float(voiceover_audio.length)
)

# Add the clip to voiceover track
voiceover_track = Track()
voiceover_track.add_clip(0, voiceover_clip)
timeline.add_track(voiceover_track)

print("✓ Voiceover track added")
print("Generating stream...")
stream_url = timeline.generate_stream()
play_stream(stream_url)

✓ Voiceover track added
Generating stream...


Now we hear the background music with the video. The video is taking shape! Let's add the final layer.

## Layer 3: Background Music

Lets add a track with some background music

In [ ]:
# Create background music clip (low volume)
music_clip = Clip(
    asset=AudioAsset(
        id=background_music.id,
        start=0,
        volume=0.15  # Low volume so it doesn't overpower voiceover
    ),
    duration= float(voiceover_audio.length)
)

# Add the clip to audio track
music_track = Track()
music_track.add_clip(0, music_clip)
timeline.add_track(music_track)

print("✓ Music track added")
print("Generating stream...")
stream_url = timeline.generate_stream()
play_stream(stream_url)

✓ Music track added
Generating stream...


## Lets create a vertical video suitable for shorts, reels, tiktok, etc.

In [ ]:
timeline.resolution=("608x1080")

vertical_stream_url = timeline.generate_stream()
play_stream(vertical_stream_url)

## Final Result

We've built a complete faceless video! Our timeline now has three layers working together:

1. **Visual Layer** - Background video (muted) provides engaging imagery
2. **Voiceoverr** - AI generated voiceover is the center piece of our video
3. **Background Audio Layer** - Background music adds polish

## Try This Next

Experiment with variations:
- **Different animations**: Try `CaptionAnimation.karaoke`, `box_highlight`, or `reveal` for caption effects
- **Multiple video cuts**: Add several video clips trimmed from different timestamps for visual variety
- **Text overlays**: Add `TextAsset` clips for titles, lower thirds, or call-to-action messages
- **Filters**: Apply `Filter.greyscale` or other effects to the video for stylistic looks
- **Position control**: Use `position` and `offset` to place elements strategically on screen

The pipeline is yours to customize—change the topic, swap assets, adjust timing, and create unique content programmatically.